In [1]:
import torch
import os
from torch import nn
from torch.nn import functional as F
from PIL import Image
from torch.utils.data import Dataset
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
# from natsort import natsorted

In [12]:
image_size = 64
batch_size = 32


In [13]:
transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    # transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [14]:
train_dataset = datasets.CelebA(root='data/', split='train', transform=transform, target_type='attr',download=True)
test_dataset = datasets.CelebA(root='data/', split='test', transform=transform, target_type='attr',download=True)

Files already downloaded and verified
Files already downloaded and verified


In [15]:
train_dataset, test_dataset

(Dataset CelebA
     Number of datapoints: 162770
     Root location: data/
     Target type: ['attr']
     Split: train
     StandardTransform
 Transform: Compose(
                Resize(size=(64, 64), interpolation=bilinear, max_size=None, antialias=None)
                ToTensor()
            ),
 Dataset CelebA
     Number of datapoints: 19962
     Root location: data/
     Target type: ['attr']
     Split: test
     StandardTransform
 Transform: Compose(
                Resize(size=(64, 64), interpolation=bilinear, max_size=None, antialias=None)
                ToTensor()
            ))

In [16]:
train_data_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True, num_workers=8, drop_last=True)
test_data_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False, num_workers=8, drop_last=True)

In [17]:
device = 'mps'

In [18]:
class ConditionalVAE(nn.Module):

    def __init__(self,
                 in_channels: int,
                 num_classes: int,
                 latent_dim: int,
                 encoder_dims: list,
                 decoder_dims: list,
                 img_size:int = 64,
                 **kwargs) -> None:
        super(ConditionalVAE, self).__init__()

        self.latent_dim = latent_dim
        self.img_size = img_size

        self.embedd_class = nn.Linear(num_classes, img_size * img_size)
        self.embed_data = nn.Conv2d(in_channels, in_channels, kernel_size=1)

        modules = []
        self.encoder = nn.ModuleList()
        in_channels += 1 # To account for the extra label channel
        for i in encoder_dims:
            self.encoder.append(
                nn.Conv2d(in_channels, out_channels=i,
                          kernel_size=3,stride=2, padding=1)
            )
            self.encoder.append(nn.LeakyReLU())

            in_channels = i

        self.encoder = nn.Sequential(*self.encoder)
        self.fc_mu = nn.Linear(encoder_dims[-1]*4, latent_dim)
        self.fc_var = nn.Linear(encoder_dims[-1]*4, latent_dim)


        # Build Decoder

        self.decoder_input = nn.Linear(latent_dim + num_classes, encoder_dims[-1] * 4)

        self.decoder = nn.ModuleList()

        decoder_in_channel = encoder_dims[-1]
        for i in range(len(decoder_dims)):

            self.decoder.append(
                nn.ConvTranspose2d(decoder_in_channel,
                                       decoder_dims[i],
                                       kernel_size=3,
                                       stride = 2,
                                       padding=1,
                                       output_padding=1),
            )

            self.decoder.append(
                nn.LeakyReLU()
            )

            decoder_in_channel = decoder_dims[i]


        self.decoder = nn.Sequential(*self.decoder)

        self.final_layer = nn.Sequential(
                            nn.Conv2d(decoder_dims[-1], out_channels= 3,
                                      kernel_size= 3, padding= 1),
                            nn.Tanh())

    def encode(self, input) :
       
        result = self.encoder(input)
        result = torch.flatten(result, start_dim=1)

        # Split the result into mu and var components
        # of the latent Gaussian distribution
        mu = self.fc_mu(result)
        log_var = self.fc_var(result)

        return [mu, log_var]

    def decode(self, z) :
        result = self.decoder_input(z)
        result = result.view(-1, 512, 2, 2)
        result = self.decoder(result)
        result = self.final_layer(result)
        return result

    def reparameterize(self, mu, logvar) :

        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return eps * std + mu

    def forward(self, input, c) :
        embedded_class = self.embedd_class(c) # Embedding 40 -> 64*64
        embedded_class = embedded_class.view(-1, self.img_size, self.img_size).unsqueeze(1) # reshape to (1,64,64)
        # embedded_input = self.embed_data(input)

        x = torch.cat([input, embedded_class], dim = 1)
        mu, log_var = self.encode(x)

        z = self.reparameterize(mu, log_var)

        z = torch.cat([z, c], dim = 1)
        return  self.decode(z), mu, log_var

    

    def sample(self,
               num_samples:int,
               current_device: int,
               **kwargs) :
        """
        Samples from the latent space and return the corresponding
        image space map.
        :param num_samples: (Int) Number of samples
        :param current_device: (Int) Device to run the model
        :return: (Tensor)
        """
        y = kwargs['labels'].float()
        z = torch.randn(num_samples,
                        self.latent_dim)

        z = z.to(current_device)

        z = torch.cat([z, y], dim=1)
        samples = self.decode(z)
        return samples

    def generate(self, x, **kwargs) :
        """
        Given an input image x, returns the reconstructed image
        :param x: (Tensor) [B x C x H x W]
        :return: (Tensor) [B x C x H x W]
        """

        return self.forward(x, **kwargs)[0]

In [19]:
model = ConditionalVAE(3, 40, 128, [32, 64, 128, 256, 512], [256, 128, 64, 32, 32])
model.to(device)

ConditionalVAE(
  (embedd_class): Linear(in_features=40, out_features=4096, bias=True)
  (embed_data): Conv2d(3, 3, kernel_size=(1, 1), stride=(1, 1))
  (encoder): Sequential(
    (0): Conv2d(4, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (1): LeakyReLU(negative_slope=0.01)
    (2): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (3): LeakyReLU(negative_slope=0.01)
    (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (5): LeakyReLU(negative_slope=0.01)
    (6): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (7): LeakyReLU(negative_slope=0.01)
    (8): Conv2d(256, 512, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (9): LeakyReLU(negative_slope=0.01)
  )
  (fc_mu): Linear(in_features=2048, out_features=128, bias=True)
  (fc_var): Linear(in_features=2048, out_features=128, bias=True)
  (decoder_input): Linear(in_features=168, out_features=2048, bias=True)
  (decoder): Sequential(
    (0): Con

In [20]:
def loss_function(input, recons, mu, log_var):
    recons_loss =F.mse_loss(recons, input)
    kld_loss = torch.mean(-0.5 * torch.sum(1 + log_var - mu ** 2 - log_var.exp(), dim = 1), dim = 0)
    loss = recons_loss + kld_loss
    return loss

def train_step(model: torch.nn.Module,
               data_loader: torch.utils.data.DataLoader,
               optimizer: torch.optim.Optimizer,
               device: torch.device = device):
    train_loss = 0
    for batch, (X, y) in enumerate(data_loader):
        # Send training data to CPU or GPU
        X = X.to(torch.float32)
        y = y.to(torch.float32)
        X, y = X.to(device), y.to(device)
        # Forward pass
        X_hat, mu, log_var = model(X,y) 

        # Calculate loss
        loss = loss_function(X, X_hat, mu, log_var)
        # print(loss)
        train_loss += loss

        # Grad of optimizer to zero (optimization the weights)
        optimizer.zero_grad()

        # Update the weights by backpropagation
        loss.backward()
        optimizer.step()

    # Calculate loss and accuracy per epoch
    train_loss /= len(data_loader)
    # train_acc /= len(data_loader)
    print(f"Train loss: {train_loss:.5f}\n")
    return train_loss.to('cpu').detach().numpy()

def test_step(data_loader: torch.utils.data.DataLoader,
              model: torch.nn.Module,
              device: torch.device = device):
    test_loss, test_acc = 0, 0
    model.eval() # put model in evaluation mode

    with torch.inference_mode(): 
        for X, y in data_loader:
            # Send testing data to CPU or GPU
            X = X.to(torch.float32)
            y = y.to(torch.float32)
            X, y = X.to(device), y.to(device)
            
            # Forward pass
            X_hat, X, mu, log_var = model(X,y)
            
            # Calculate loss and accuracy
            test_loss += loss_function(X, X_hat, mu, log_var)
    
        test_loss /= len(data_loader)
        # test_acc /= len(data_loader)
        print(f"Test loss: {test_loss:.5f}\n")
        return test_loss.detach().numpy()

In [21]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)

In [17]:
epochs = 20

train_plot = []
test_plot = []

for epoch in range(epochs):
    print(f"Epoch: {epoch}\n---------")
    train_loss, train_acc = train_step(data_loader=train_data_loader, 
        model=model, 
        optimizer=optimizer
    )
    
    train_plot.append((train_acc, train_loss))

    test_loss, test_acc = test_step(data_loader=test_data_loader,
        model=model,
    )
    test_plot.append((test_acc, test_loss))

Epoch: 0
---------


KeyboardInterrupt: 